In [5]:
import numpy as np;
import os;
import util;
import visualize;

def comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server,img_caption_pre=None):
    img_files_all=[];
    captions_all=[];
    if img_caption_pre is not None:
        assert len(img_caption_pre)==len(img_dirs);
    batch_str=True;
    if range_batches is None:
        range_batches=range(1);
        batch_str=False;
    
    for batch_num in range_batches:
    # range(1,num_batches+1):
        for im_num in range_images:
            for idx_img_dir,img_dir in enumerate(img_dirs):
                if loss_post is None:
                    loss_str='';
                else:
                    loss_all=np.load(os.path.join(img_dir,str(batch_num)+loss_post));
                    if im_num>loss_all.shape[0]:
                        continue;
                    loss_curr=loss_all[im_num-1,0];
                    loss_str="{:10.4f}".format(loss_curr);
                if not batch_str:
                    files_curr=[os.path.join(img_dir,str(im_num)+file_post_curr) for file_post_curr in file_post];
                else:
                    files_curr=[os.path.join(img_dir,str(batch_num)+'_'+str(im_num)+file_post_curr) for file_post_curr in file_post];
                files_curr=[util.getRelPath(file_curr,dir_server) for file_curr in files_curr];
                captions_curr=[os.path.split(file_curr)[1]+' '+loss_str for file_curr in files_curr];
                if img_caption_pre is not None:
                    captions_curr=[img_caption_pre[idx_img_dir]+' '+caption_curr for caption_curr in captions_curr];
                img_files_all.append(files_curr);
                captions_all.append(captions_curr);
    visualize.writeHTML(out_file_html,img_files_all,captions_all,224,224);

In [22]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05'
range_batches=range(1,2);
# batch_size=60;
range_images=range(1,100);
img_dir_meta=os.path.join(dir_meta,'over_time_train');
range_models=range(75,5000,75*8);
img_dir=[os.path.join(img_dir_meta,str(range_model_curr)) for range_model_curr in range_models]

file_post=['_org.jpg','_warp.jpg'];
out_file_html=img_dir_meta+'.html';
print out_file_html
img_caption_pre=[str(model_num) for model_num in range_models];
comparativeLossViz(img_dir,file_post,None,None,range_images,out_file_html,dir_server,img_caption_pre)


/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/over_time_train.html


In [15]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/horse_project/tps_train_allKP_adam_noBad'

dir_meta='/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'
dir_loss=os.path.join(dir_meta,'intermediate');
loss_pre='loss_all_';
loss_post_val='_val.npy';
loss_post_normal='.npy';
test_after=30;

loss_files=[file_curr for file_curr in os.listdir(dir_loss) if file_curr.startswith(loss_pre)];
loss_nums=[int(file_curr[:file_curr.rindex('.')].split('_')[2]) for file_curr in loss_files];
print min(loss_nums),max(loss_nums);
loss_num=max(loss_nums);
loss_file=os.path.join(dir_loss,loss_pre+str(loss_num)+loss_post_normal);
loss_file_val=os.path.join(dir_loss,loss_pre+str(loss_num)+loss_post_val);
loss=np.load(loss_file);
loss_val=np.load(loss_file_val);
iter_loss=range(loss.size);
iter_val=range(0,loss_val.size*test_after,test_after);
loss_val_new=[loss_curr for loss_curr in loss_val if loss_curr<1];
iter_val=[loss_curr for idx,loss_curr in enumerate(iter_val) if loss_val[idx]<1];
loss_val=loss_val_new;

out_file_score=os.path.join(dir_meta,'loss.png');
print out_file_score.replace(dir_server,'vision1.idav.ucdavis.edu:1000');
visualize.plotSimple([(iter_loss,loss),(iter_val,loss_val)],\
                     out_file_score,title='Score Loss at '+str(iter_loss[-1]),xlabel='Iterations',\
                     ylabel='Loss',legend_entries=['Train','Val'])




300 2100
vision1.idav.ucdavis.edu:1000/temp/tps_train_allKP_adam_noBad_bn/loss.png


In [20]:
import random;
# horse_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP.txt';
# human_path='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP.txt';

horse_path = '/home/SSD3/maheen-data/horse_project/horse_resize/matches_5_train_fiveKP.txt';
human_path = '/home/SSD3/maheen-data/horse_project/aflw/matches_5_train_fiveKP_noIm.txt';

horse_data=util.readLinesFromFile(horse_path);
human_data=util.readLinesFromFile(human_path);
idx_keep=range(0,len(horse_data),5);
print len(horse_data);
print len(idx_keep);
horse_data=[horse_data[idx] for idx in idx_keep];
human_data=[human_data[idx] for idx in idx_keep];
shuffle_idx=range(len(horse_data));
random.shuffle(shuffle_idx);
horse_data=[horse_data[idx] for idx in shuffle_idx];
human_data=[human_data[idx] for idx in shuffle_idx];

for file_curr,data_curr in [(horse_path,horse_data),(human_path,human_data)]:
    out_file_curr=file_curr[:file_curr.rindex('.')]+'_select.txt';
    print out_file_curr;
    util.writeFile(out_file_curr,data_curr);
    







4720
944
/home/SSD3/maheen-data/horse_project/horse_resize/matches_5_train_fiveKP_select.txt
/home/SSD3/maheen-data/horse_project/aflw/matches_5_train_fiveKP_noIm_select.txt


In [21]:
# path_models='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/intermediate/'
dir_meta='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05'
path_to_models=os.path.join(dir_meta,'intermediate')
out_dir_meta=os.path.join(dir_meta,'over_time_train');
util.mkdir(out_dir_meta);

model_pre=os.path.join(path_to_models,'model_all_');
model_post='.dat'
range_models=range(75,5000,75*8);
print range_models

path_to_th='/home/maheenrashid/Downloads/horses/torch/test_xiuye.th';
out_file_sh=out_dir_meta+'.sh'
commands=[];
for model_idx,model_num in enumerate(range_models):
    model_curr=model_pre+str(model_num)+model_post;
    assert os.path.exists(model_curr);
    out_dir_curr=os.path.join(out_dir_meta,str(model_num));
    command=['th',path_to_th];
    command.extend(['-kp_net_file',model_curr]);
    command.extend(['out_dir',out_dir_curr]);
    command=' '.join(command);
    commands.append(command);
# print commands
util.writeFile(out_file_sh,commands)
print out_file_sh
    



[75, 675, 1275, 1875, 2475, 3075, 3675, 4275, 4875]
/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/over_time_train.sh
